## Image Background removal with $U^2$-Net and OpenVINO

### Import the PyTorch Library and $U^2$-Net

In [ ]:
import os, sys
import time
from collections import namedtuple
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from IPython.display import HTML, FileLink, display
from openvino.runtime import Core

sys.path.append(os.pardir)
from utils.models.u2net import U2NET, U2NETP

### Settings

In [ ]:
IMAGE_DIR = "data"
model_config = namedtuple("ModelConfig", ["name", "url", "model", "model_args"])

u2net_lite = model_config(
    name="u2net_lite",
    url="https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy",
    model=U2NETP,
    model_args=(),
)
u2net = model_config(
    name="u2net",
    url="https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ",
    model=U2NET,
    model_args=(3, 1),
)
u2net_human_seg = model_config(
    name="u2net_human_seg",
    url="https://drive.google.com/uc?id=1-Yg0cxgrNhHP-016FPdp902BR-kSsA4P",
    model=U2NET,
    model_args=(3, 1),
)

# Set u2net_model to one of the three configurations listed above.
u2net_model = u2net_lite

In [ ]:
# The filenames of the downloaded and converted models.
MODEL_DIR = "model"
model_path = Path(MODEL_DIR) / u2net_model.name / Path(u2net_model.name).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

In [ ]:
### debug
print(model_path)
print(onnx_path)
print(ir_path)

### Load the $U^2$-Net model

In [ ]:
if not model_path.exists():
    import gdown

    os.makedirs(name=model_path.parent, exist_ok=True)
    print("Start downloadig model weights file...")
    with open(model_path, "wb") as model_file:
        gdown.download(url=u2net_model.url, output=model_file)
        print(f"Model weights have been downloaded to {model_path}")

In [ ]:
# Load the model
net = u2net_model.model(*u2net_model.model_args)
net.eval()

# Load the weights.
print(f"Loading model weights from: '{model_path}'")
net.load_state_dict(state_dict=torch.load(model_path, map_location="cpu"))

# Save the model if it does not exist yet.
if not model_path.exists():
    print("\nSaving the model")
    torch.save(obj=net.state_dict(), f=str(model_path))
    print(f"Model saved at {model_path}")

### Convet PyTorch $U^2$-Net model to ONNX

In [ ]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, 512, 512)
    torch.onnx.export(model=net, args=dummy_input, f=onnx_path, opset_version=11)
    print(f"ONNX model exported to {onnx_path}")
else:
    print(f"ONNX model {onnx_path} already exists")

### Convert ONNX model to OpenVINO IR Format

In [ ]:
# Construct the command for Model Optimizer
# Set log_level to CRITICAL to suppress warnings that can be ignored for this demo
mo_command = f"""mo
                 --input_model "model/u2net_lite/u2net_lite.onnx"
                 --input_shape "[1,3,512,512]"
                 --mean_values="[123.675,116.28,103.53]"
                 --scale_values="[58.395,57.12,57.375]"
                 --data_type FP16
                 --output_dir "model/u2net_lite"
                 --log_level "CRITICAL"
                 """

mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO")
print(mo_command)


In [ ]:
import subprocess
if not ir_path.exists():
    print("Exporting ONNX mdoel to IR... This may take a few minutes")
    output = subprocess.check_output(mo_command, shell=True)
    print (output.decode('utf-8'))
    # mo_result = %sx $mo_command
    # print("\n".join(mo_result))
else:
    print(f"IR model {ir_path} already exists")


### Load and Pre-Process input Image

In [ ]:
IMAGE_PATH = Path(IMAGE_DIR) / "coco_dog.jpg"

image = cv2.cvtColor(
    src=cv2.imread(filename=str(IMAGE_PATH)),
    code=cv2.COLOR_BGR2RGB
)   

resized_image = cv2.resize(image, (512, 512))

# Convert the image shape to a shape and a data type expected by the network
# for OpenVINO IR model: (1, 3, 512, 512)
input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)

In [ ]:
input_image.shape

### Do inference on OpenVINO IR Model

In [ ]:
# Load the network to OpenVINO Runtime.
ie = Core()
model_ir = ie.read_model(model=ir_path)
compiled_model_ir = ie.compile_model(model=model_ir, device_name="CPU")

# Get the names of input and output layers
input_layer_ir = compiled_model_ir.input(0)
output_layer_ir = compiled_model_ir.output(0)

# De inference on the input image
start_time = time.perf_counter()
result = compiled_model_ir([input_image])[output_layer_ir]
ent_time = time.perf_counter()

### Visualize Results

In [ ]:
"""
Resize the network result to the image shpae and round the values
to 0 (background) and 1 (foreground).
The network result has (1, 1, 512, 512) shape. The `np.squeeze` function converts this to (512, 512)
"""

resized_result = np.rint(
    cv2.resize(src=np.squeeze(result), dsize=(image.shape[1], image.shape[0]))
).astype(np.uint8)

# Create a copy of the image and set all background values to 255 (white).
bg_removed_result = image.copy()
bg_removed_result[resized_result == 0] = 255

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 7))
ax[0].imshow(image)
ax[1].imshow(resized_result, camp="gray")
ax[2].imshow(bg_removed_result)
for a in ax:
    a.axis("off")

### Add a New Background Image

In [ ]:
BACKGROUND_FILE = "./data/wall.jpg"
OUTPUT_DIR = "output"

os.makedirs(name=OUTPUT_DIR, exist_ok=True)

background_image = cv2.cvtColor(src=cv2.imread(BACKGROUND_FILE), code=cv2.COLOR_BGR2RGB)
background_resize = cv2.resize(background_image, (image.shape[1], image.shape[0]))

# Set all the foreground pixels from the result to 0
# in the background image and the image with the background removed.
background_image[resized_result == 1] = 0
new_image = background_image + bg_removed_result

# Save the generated image.
new_image_path = Path(f"{OUTPUT_DIR}/{IMAGE_PATH.stem}-{Path(BACKGROUND_FILE).stem}.jpg")
cv2.imwrite(filename=str(new_image_path), img=cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR))

# Display the original image and the image with the new background side by side
fig, ax = plt.subplots(1, 2, figsize=(18, 7))
ax[0].imshow(image)
ax[1].imshow(new_image)
for a in ax:
    a.axis("off")
plt.show()

# Create a link to download the image
image_link = FileLink(new_image_path)
image_link.html_link_str = "<a href='%s' download>%s</a>"
display(
    HTML(
        f"The generated image <code>{new_image_path.name}</code> is saved in"
        f"the directory <code>{new_image_path.parent}</code>. You can also"
        f"{image_link._repr_html_()}"
    )
)
